[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sci-ndp/ndp-ep-py/blob/main/docs/source/tutorials/getting_started.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sci-ndp/ndp-ep-py/main?filepath=docs%2Fsource%2Ftutorials%2Fgetting_started.ipynb)

# Getting Started with NDP EP Python Client

Welcome to the interactive tutorial for the NDP EP Python client library!

## What is NDP EP?

The National Data Platform (NDP) EP allows you to:
- 🏢 Manage organizations and datasets
- 📊 Register data resources (URLs, S3, Kafka topics)
- 🔍 Search and discover datasets
- 📈 Monitor system health

In [ ]:
# Install the ndp-ep library
!pip install ndp-ep

# Import required libraries
import json
from pprint import pprint
from ndp_ep import APIClient

print("✅ Libraries installed successfully!")

✅ Libraries installed successfully!


## 1. Setting Up the Client

**Authentication Options:**
1. **API Token** (Recommended): Get from https://nationaldataplatform.org/
2. **Username/Password**: Use your NDP credentials
3. **No Auth**: Limited to public endpoints

In [12]:
# Configuration
API_BASE_URL = "http://localhost:8003"

# Option 1: Using API token (uncomment to use)
# API_TOKEN = "your-api-token-here"
# client = APIClient(base_url=API_BASE_URL, token=API_TOKEN)

# Option 2: Using username/password (uncomment to use)
# USERNAME = "your-username"
# PASSWORD = "your-password"
# client = APIClient(base_url=API_BASE_URL, username=USERNAME, password=PASSWORD)

# Option 3: Demo mode (limited functionality)
try:
    client = APIClient(base_url=API_BASE_URL)
    print("✅ Client initialized successfully!")
    print(f"🌐 Connected to: {API_BASE_URL}")
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")
    print("💡 Tip: Make sure the API endpoint is accessible")

✅ Client initialized successfully!
🌐 Connected to: http://localhost:8003


## 2. Exploring Organizations

Organizations are containers for datasets and resources.

In [13]:
# List organizations from different servers
servers = ["local", "global", "pre_ckan"]

print("🏢 Listing Organizations...")
for server in servers:
    try:
        orgs = client.list_organizations(server=server)
        print(f"\n📋 Server '{server}': {len(orgs)} organizations")
        
        # Show first few organizations
        for i, org in enumerate(orgs[:3]):
            print(f"   {i+1}. {org}")
        
        if len(orgs) > 3:
            print(f"   ... and {len(orgs) - 3} more")
            
    except Exception as e:
        print(f"   ❌ Error with {server}: {e}")

🏢 Listing Organizations...

📋 Server 'local': 5 organizations
   1. research_tutorial_20250628_230121
   2. research_tutorial_20250630_112306
   3. test_org
   ... and 2 more

📋 Server 'global': 46 organizations
   1. __
   2. archives-of-the-california-wildfire-forest-resilience-task-force
   3. burnpro3d
   ... and 43 more
   ❌ Error with pre_ckan: Error listing organizations: Pre-CKAN is disabled and cannot be used.


## 3. Searching for Datasets

Search is one of the most powerful features.

In [14]:
# Simple search examples
search_terms = [
    ["climate"],
    ["weather", "temperature"],
    ["data"],
    ["sensor"]
]

print("🔍 Performing Searches...")
for terms in search_terms:
    try:
        results = client.search_datasets(terms=terms, server="global")
        print(f"\n🔎 Search for {terms}: {len(results)} results")
        
        # Show first result details
        if results:
            first = results[0]
            print(f"   📄 Example: {first.get('title', 'No title')}")
            print(f"   🏢 Organization: {first.get('organization', {}).get('title', 'Unknown')}")
            print(f"   📊 Resources: {len(first.get('resources', []))}")
            
    except Exception as e:
        print(f"   ❌ Search failed for {terms}: {e}")

print("\n✅ Search exploration completed!")

🔍 Performing Searches...

🔎 Search for ['climate']: 57 results
   📄 Example: Climate and Tectonic Controls on Bedrock Hillslopes, CA 2019
   🏢 Organization: Unknown
   📊 Resources: 2

🔎 Search for ['weather', 'temperature']: 9 results
   📄 Example: High-accuracy gas, pressure, humidity and temperature sensors: bme680
   🏢 Organization: Unknown
   📊 Resources: 1

🔎 Search for ['data']: 1000 results
   📄 Example: SANDAG Data
   🏢 Organization: Unknown
   📊 Resources: 1

🔎 Search for ['sensor']: 56 results
   📄 Example: Rainfall Sensor: rg-15
   🏢 Organization: Unknown
   📊 Resources: 1

✅ Search exploration completed!


## 4. Advanced Search

Use filters and advanced queries for more precise results.

In [15]:
print("🔍 Advanced Search Examples...")

# Advanced search with filters
advanced_searches = [
    {
        "search_term": "climate,weather",
        "server": "global"
    },
    {
        "dataset_name": "sensor", 
        "server": "global"
    },
    {
        "search_term": "data",
        "filter_list": ["format:CSV"],
        "server": "global"
    }
]

for i, search_data in enumerate(advanced_searches, 1):
    try:
        print(f"\n🔎 Advanced Search #{i}:")
        print(f"   Query: {search_data}")
        
        results = client.advanced_search(search_data)
        print(f"   Results: {len(results)} datasets found")
        
        # Show sample results
        for j, result in enumerate(results[:2]):
            title = result.get('title', result.get('name', 'Untitled'))[:50]
            print(f"      {j+1}. {title}...")
            
    except Exception as e:
        print(f"   ❌ Advanced search failed: {e}")

print("\n✅ Advanced search completed!")

🔍 Advanced Search Examples...

🔎 Advanced Search #1:
   Query: {'search_term': 'climate,weather', 'server': 'global'}
   Results: 19 datasets found
      1. Public View - Interagency Remote Automatic Weather...
      2. NOAA/National Weather Service (NWS) WATCHES, WARNI...

🔎 Advanced Search #2:
   Query: {'dataset_name': 'sensor', 'server': 'global'}
   Results: 0 datasets found

🔎 Advanced Search #3:
   Query: {'search_term': 'data', 'filter_list': ['format:CSV'], 'server': 'global'}
   Results: 0 datasets found

✅ Advanced search completed!


## 5. System Information

Check system health and get connection details.

In [16]:
print("🏥 System Information...")

# Check system status
try:
    status = client.get_system_status()
    print("\n✅ System Status:")
    pprint(status)
except Exception as e:
    print(f"\n⚠️ System status: {e}")

# Get system metrics
try:
    metrics = client.get_system_metrics()
    print("\n📊 System Metrics:")
    pprint(metrics)
except Exception as e:
    print(f"\n⚠️ System metrics: {e}")

# Get Kafka details
try:
    kafka_details = client.get_kafka_details()
    print("\n📡 Kafka Details:")
    pprint(kafka_details)
except Exception as e:
    print(f"\n⚠️ Kafka details: {e}")

print("\n💡 Note: Some endpoints require authentication")

🏥 System Information...

✅ System Status:
{'ckan_is_active_global': True,
 'ckan_is_active_local': True,
 'ckan_local_enabled': True,
 'keycloak_is_active': True}

📊 System Metrics:
{'cpu': '2.1%',
 'disk': '1.7%',
 'memory': '13.8%',
 'public_ip': '161.111.139.206',
 'services': {'ckan_is_active_global': True,
              'ckan_is_active_local': True,
              'ckan_local_enabled': True,
              'keycloak_is_active': True}}

📡 Kafka Details:
{'kafka_connection': True,
 'kafka_host': '155.101.6.194',
 'kafka_port': 9092,
 'kafka_prefix': 'data_stream_',
 'max_streams': 10}

💡 Note: Some endpoints require authentication


## 6. Creating Resources (Requires Authentication)

**⚠️ Note**: The following examples require valid credentials.
Without authentication, you'll see error messages (which is expected).

In [18]:
# Authentication setup - API Token input
import getpass

# Secure token input (won't be visible on screen)
print("🔐 Please enter your API token:")
print("(The token will not be displayed for security)")
API_TOKEN = getpass.getpass("API Token: ")

# Configure authenticated client
try:
    client = APIClient(base_url=API_BASE_URL, token=API_TOKEN)
    print("✅ Client configured with API token successfully!")
    print(f"🌐 Connected to: {API_BASE_URL}")
except Exception as e:
    print(f"❌ Failed to configure client: {e}")
    print("💡 Please check your token and try again")

🔐 Please enter your API token:
(The token will not be displayed for security)
✅ Client configured with API token successfully!
🌐 Connected to: http://localhost:8003


In [19]:
import time

# Example organization data
org_data = {
    "name": f"demo_org_{int(time.time())}",  # Unique name
    "title": "Demo Organization",
    "description": "A demonstration organization"
}

print("🏢 Creating Organization (requires auth)...")
print(f"📝 Data: {org_data}")

try:
    result = client.register_organization(org_data)
    print(f"\n✅ Organization created: {result}")
    created_org = org_data["name"]
except Exception as e:
    print(f"\n❌ Expected error (no auth): {e}")
    created_org = "demo_org"  # Fallback for examples

🏢 Creating Organization (requires auth)...
📝 Data: {'name': 'demo_org_1751968397', 'title': 'Demo Organization', 'description': 'A demonstration organization'}

✅ Organization created: {'id': '382fdb42-6677-4500-b6ce-df5ecb866455', 'message': 'Organization created successfully'}


In [20]:
# Example URL resource
url_resource = {
    "resource_name": f"climate_data_{int(time.time())}",
    "resource_title": "Climate Data CSV",
    "owner_org": created_org,
    "resource_url": "https://example.com/climate.csv",
    "file_type": "CSV",
    "notes": "Monthly climate data"
}

print("🌐 Registering URL Resource (requires auth)...")
print(f"📝 Data: {url_resource}")

try:
    result = client.register_url(url_resource)
    print(f"\n✅ URL resource created: {result}")
except Exception as e:
    print(f"\n❌ Expected error (no auth): {e}")

🌐 Registering URL Resource (requires auth)...
📝 Data: {'resource_name': 'climate_data_1751968410', 'resource_title': 'Climate Data CSV', 'owner_org': 'demo_org_1751968397', 'resource_url': 'https://example.com/climate.csv', 'file_type': 'CSV', 'notes': 'Monthly climate data'}

✅ URL resource created: {'id': 'f5869b5e-4ed2-48ca-9c0e-624ed0cee0a8'}


In [21]:
# Example S3 resource
s3_resource = {
    "resource_name": f"s3_dataset_{int(time.time())}",
    "resource_title": "S3 Dataset",
    "owner_org": created_org,
    "resource_s3": "s3://my-bucket/dataset.parquet",
    "notes": "Large dataset in S3"
}

print("☁️ Registering S3 Resource (requires auth)...")
print(f"📝 Data: {s3_resource}")

try:
    result = client.register_s3_link(s3_resource)
    print(f"\n✅ S3 resource created: {result}")
except Exception as e:
    print(f"\n❌ Expected error (no auth): {e}")

☁️ Registering S3 Resource (requires auth)...
📝 Data: {'resource_name': 's3_dataset_1751968425', 'resource_title': 'S3 Dataset', 'owner_org': 'demo_org_1751968397', 'resource_s3': 's3://my-bucket/dataset.parquet', 'notes': 'Large dataset in S3'}

✅ S3 resource created: {'id': 'ad410b3f-08b2-422b-b16d-fc861a04f88b'}


## 7. Error Handling Best Practices

In [22]:
def safe_api_call(func, *args, description="API call", **kwargs):
    """
    Safely execute an API call with comprehensive error handling.
    """
    try:
        print(f"🔄 Attempting: {description}...")
        result = func(*args, **kwargs)
        print(f"✅ Success: {description}")
        return result
    except ValueError as e:
        print(f"❌ API Error in {description}: {e}")
        return None
    except ConnectionError as e:
        print(f"🌐 Connection Error in {description}: {e}")
        return None
    except Exception as e:
        print(f"⚠️ Unexpected error in {description}: {e}")
        return None

# Examples of safe API calls
print("🛡️ Demonstrating Safe API Calls...")

# Safe organization listing
orgs = safe_api_call(
    client.list_organizations,
    server="global",
    description="listing organizations"
)

# Safe search
results = safe_api_call(
    client.search_datasets,
    ["test"],
    server="global",
    description="searching datasets"
)

print("\n✅ Safe API call demonstrations completed!")

🛡️ Demonstrating Safe API Calls...
🔄 Attempting: listing organizations...
✅ Success: listing organizations
🔄 Attempting: searching datasets...
✅ Success: searching datasets

✅ Safe API call demonstrations completed!
